Importing libraries

In [1]:
# For fetching citybik API
import requests

import numpy as np
import pandas as pd

from scipy.spatial.distance import cdist

Fetching the API

## Citybik.es exercise

In [2]:
citybik_api_url = "http://api.citybik.es/v2/networks/to-bike"

response = requests.get(citybik_api_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:

    # Parse and use the response data (e.g., JSON)
    data = response.json()
else:
    # Handle the error
    print(f"Request failed with status code: {response.status_code}")

2. Count and print the number of active stations (a station is active if its extra.status field is "online").

In [3]:
active_stations = sum([1 if station['extra']['status'] == 'online' else 0 for station in data['network']['stations']])
print(f"Active stations: {active_stations}")

Active stations: 9


3. Count and print the total number of bikes available (field free_bikes) and the number of free docks
(field empty_slots) throughout all stations.

In [4]:
total_bikes = 0
total_empty_slots = 0

for station in data['network']['stations']:
    total_bikes += station['free_bikes']
    total_empty_slots += station['empty_slots']

print(f"Total bikes: {total_bikes}")
print(f"Total empty slots: {total_empty_slots}")   

Total bikes: 104
Total empty slots: 94


Given the coordinates (latitude, longitude) of a point (e.g. 45.074512, 7.694419), identify the
closest bike station to it that has available bikes. For computing the distance among two points
(given their coordinates), you can use the function distance_coords() defined in the code snippet
below (which is an implementation of the great-circle distance):
from math import cos, acos, sin
from math import cos, acos, sin

def distance_coords(lat1, lng1, lat2, lng2):
    """Compute the distance among two points."""
    deg2rad = lambda x: x * 3.141592 / 180
    lat1, lng1, lat2, lng2 = map(deg2rad, [ lat1, lng1, lat2, lng2 ])
    R = 6378100 # Radius of the Earth, in meters
    return R * acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lng1 - lng2))

In [5]:
from math import cos, acos, sin
def distance_coords(lat1, lng1, lat2, lng2):
    """Compute the distance among two points."""
    deg2rad = lambda x: x * 3.141592 / 180
    lat1, lng1, lat2, lng2 = map(deg2rad, [ lat1, lng1, lat2, lng2 ])
    R = 6378100 # Radius of the Earth, in meters
    return R * acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lng1 - lng2))

Mode 1

In [6]:
given_coords = (45.074512, 7.694419)
arg_min = 0
min_distance = distance_coords(given_coords[0], given_coords[1], data['network']['stations'][0]['latitude'], data['network']['stations'][0]['longitude'])
for i, station in enumerate(data['network']['stations']):
    current_distance = distance_coords(given_coords[0], given_coords[1], station['latitude'], station['longitude'])
    if min_distance > current_distance:
        arg_min = i
        min_distance = distance_coords(given_coords[0], given_coords[1], station['latitude'], station['longitude'])

data['network']['stations'][arg_min]

{'empty_slots': 4,
 'extra': {'number': 10,
  'reviews': 40,
  'score': 4.5,
  'status': 'online',
  'uid': '592'},
 'free_bikes': 0,
 'id': 'f16cba9c6dc917e833eab7bf3e0168b7',
 'latitude': 45.114828943536864,
 'longitude': 7.641313076019287,
 'name': '10. Gallo Praile',
 'timestamp': '2023-10-20T13:09:10.468000Z'}

Mode 2

In [7]:
min(data['network']['stations'], key=lambda station: distance_coords(given_coords[0], given_coords[1], station['latitude'], station['longitude']))

{'empty_slots': 4,
 'extra': {'number': 10,
  'reviews': 40,
  'score': 4.5,
  'status': 'online',
  'uid': '592'},
 'free_bikes': 0,
 'id': 'f16cba9c6dc917e833eab7bf3e0168b7',
 'latitude': 45.114828943536864,
 'longitude': 7.641313076019287,
 'name': '10. Gallo Praile',
 'timestamp': '2023-10-20T13:09:10.468000Z'}

## MNIST exercise

3.2

In [8]:
def print_mnist_digit(k):
    digit_to_print = np.loadtxt('../datasets/mnist_test.csv', delimiter=',', dtype=float)[k - 1]
    for i in range(28):
        for j in range(28):
            pixel = digit_to_print[i * 28 + j]
            if pixel >= 0 and pixel < 64:
                print(' ', end='')
            elif pixel >= 64 and pixel < 128:
                print('.', end='')
            elif pixel >= 128 and pixel < 192:
                print('*', end='')
            elif pixel >= 192 and pixel < 256:
                print('#', end='')
        print('')
    

In [9]:
print_mnist_digit(130)

                            
                            
                            
                            
                            
               .#      **   
              .##..*#####   
             #########*.    
            #####***.       
           ##*              
          *##               
          ##                
         .##                
          ###*              
          .#####.           
             *###*          
               *###*        
                 ###        
                 .##        
                 ###        
               .###         
         .    *###.         
        .# .*###*           
        .######.            
         *##*.              
                            
                            
                            


3.3

In [10]:
positions = np.array([26, 30, 32, 35]) - 1
digits_to_compute_distance = np.loadtxt('../datasets/mnist_test.csv', delimiter=',', dtype=float)[positions]

In [11]:
cdist(digits_to_compute_distance, digits_to_compute_distance, metric='euclidean')

array([[   0.        , 3539.22336113, 3556.42011017, 3223.21454452],
       [3539.22336113,    0.        , 1171.82933911, 2531.01047015],
       [3556.42011017, 1171.82933911,    0.        , 2515.56713287],
       [3223.21454452, 2531.01047015, 2515.56713287,    0.        ]])

3.4. Based on the distances computed in the previous step and knowing that the digits listed in
Exercise 3 are (not necessarily in this order) 7, 0, 1, 1, can you assign the correct label to each of the
digits of Exercise 3?

We find that the second and third digit have minimum distance with regard to others, we could guess that these are the digits "1".
Let's print them to be sure:

In [12]:
print_mnist_digit(positions[1] + 1)
print_mnist_digit(positions[2] + 1)

                            
                            
                            
                            
            ##              
            ##.             
            ###             
            .##             
             .#*            
             .##            
             ###            
             .##            
             *##            
             ###.           
              *#*           
              .##           
              .##           
              .##           
              .##           
              ###           
              ###.          
              .##.          
              .##.          
              .##           
                            
                            
                            
                            
                            
                            
                            
                            
                            
             ##             
             #

Now, if we want to guess who's "0" and who's "7", we could say: The arabic symbol "7" is more similar to "1" than "0", so we expect that the distance between "1" and "7" be less than "0" and "7".
Ergo, we suppose that fourth digit is "7" and first is "0", let's check

In [13]:
print("First digit:")
print_mnist_digit(positions[0] + 1)

print("Fourth digit:")
print_mnist_digit(positions[3] + 1)

First digit:
                            
                            
                            
                            
               .#....       
               ######*      
               ########     
          *##. *#######     
         *###* .########    
        *#####   *#*####.   
       *######.      ###*   
      .#######        ###   
      .######         ###.  
      ######*        ####.  
      #####*         ####.  
      #####        #####*   
      #####     .*######.   
      #####   *########.    
      *################     
      .###############      
       #############.       
        ###########.        
         #######*.          
          .##*..            
                            
                            
                            
                            
Fourth digit:
                            
                            
                            
                            
                            
                

3.5

In [14]:
dataset = pd.read_csv('../datasets/mnist_test.csv', delimiter=',', dtype=float, header=None)

In [15]:
# 1. We filter the dataset to get only the rows belonging to the digits 0 and 1
zeros = dataset[dataset[0] == 0]
ones = dataset[dataset[0] == 1]

In [16]:
# 2. We compute which pixels are black for each digit and count how many times each pixel is black
counting_black_pixels_0 = zeros.apply(lambda x: x >= 128).sum()
counting_black_pixels_1 = ones.apply(lambda x: x >= 128).sum()

In [17]:
# Due to the fact that the first column is the label, we remove it
counting_black_pixels_0 = np.array(counting_black_pixels_0[1:])
counting_black_pixels_1 = np.array(counting_black_pixels_1[1:])

In [18]:
# 3. We compute the absolute difference between the number of black pixels for each digit
z_minus_o = np.abs(counting_black_pixels_0 - counting_black_pixels_1)

In [19]:
# 4. The best pixel for separating the two digits is the one with the highest absolute difference
best_pixel_for_separing = np.argmax(z_minus_o)

In [20]:
def grid_location(k):
    return (k // 28, k % 28)

In [21]:
print(f"The best pixel for separating the two digits is {best_pixel_for_separing} located at {grid_location(best_pixel_for_separing)}")

The best pixel for separating the two digits is 406 located at (14, 14)


This is somethig that we could expect due to the zero does not have pixels in the middle of the digit and the one does.